In [1]:
import pandas as pd

import json
import glob
import os
import re

In [2]:
base = "../exp/"
paths = glob.glob(base + "building_blocks/*.json")
data = [json.load(open(path)) for path in paths]

In [3]:
queries = pd.DataFrame.from_records([{
    'features': run['feature_flags'],
    **algo }
    for run in data for exp in run.get('experiments', []) for algo in exp['algo_runs'] if algo.get('algo') in ['CH Potentials Query', 'Dijkstra Query']])

In [4]:
queries['BCC'] = ~queries['features'].str.contains('CHPOT_NO_BCC')
queries['Skip Deg. 2'] = ~queries['features'].str.contains('CHPOT_NO_DEG2')
queries['Skip Deg. 3'] = ~queries['features'].str.contains('CHPOT_NO_DEG3')

In [5]:
def heuristic_name(feature_flags):
    if 'CHPOT_ONLY_TOPO' in feature_flags:
        return 'Zero'
    elif 'CHPOT_ORACLE' in feature_flags:
        return 'Oracle'
    elif 'CHPOT_ALT' in feature_flags:
        return 'ALT'
    else:
        return 'CH'

queries['Heuristic'] = queries['features'].apply(heuristic_name)

In [7]:
table = queries.groupby(['algo', 'Heuristic', 'BCC', 'Skip Deg. 2', 'Skip Deg. 3'])[['running_time_ms', 'num_queue_pushs']].mean()
table['num_queue_pushs'] = table['num_queue_pushs'] / 1000
table = table.round(1)

table.reindex(['Zero', 'ALT', 'CH', 'Oracle'], level=1)

running_time_ms  \
algo                Heuristic BCC   Skip Deg. 2 Skip Deg. 3                    
CH Potentials Query Zero      False False       False                 1724.4   
                              True  False       False                 1253.1   
                                    True        False                  713.6   
                                                True                   558.9   
                    ALT       False False       False                  279.5   
                              True  False       False                  217.3   
                                    True        False                  117.3   
                                                True                    88.3   
                    CH        False False       False                   50.6   
                              True  False       False                   36.3   
                                    True        False                   18.8   
                                                True                    15.7   
                    Oracle    False False       False                   32.0   
                              True  False       False                   23.9   
                                    True        False                   11.8   
                                                True                     9.5   
Dijkstra Query      Zero      False False       False                 1981.2   
                    ALT       True  True        True                   293.1   
                    CH        True  True        True                    48.2   
                    Oracle    True  True        True                    33.3   

                                                             num_queue_pushs  
algo                Heuristic BCC   Skip Deg. 2 Skip Deg. 3                   
CH Potentials Query Zero      False False       False                 8120.7  
                              True  False       False                 6326.5  
                                    True        False                 2915.7  
                                                True                  1689.8  
                    ALT       False False       False                  859.4  
                              True  False       False                  684.1  
                                    True        False                  301.3  
                                                True                   178.4  
                    CH        False False       False                  138.0  
                              True  False       False                  114.0  
                                    True        False                   42.1  
                                                True                    26.0  
                    Oracle    False False       False                  138.0  
                              True  False       False                  114.0  
                                    True        False                   42.1  
                                                True                    26.0  
Dijkstra Query      Zero      False False       False                 7887.2  
                    ALT       True  True        True                   859.4  
                    CH        True  True        True                   138.0  
                    Oracle    True  True        True                   138.0

In [8]:
table.reindex(['Zero', 'CH', 'Oracle'], level=1).reset_index(level=0,drop=True).reset_index(level=[2,3]).to_latex(escape=False).split("\n")[17]

'Zero & False &        False &        False &           1981.2 &           7887.2 \\\\'